In [1]:
import sys
import os
sys.path.append(os.path.abspath(".."))
import yaml
from typing import Dict, Any
from typing import Annotated, Literal
import json
from src.utils.json_utils import repair_json_output
from langchain_core.messages import AIMessage, HumanMessage
from langchain.chat_models import init_chat_model

from mermaid import Mermaid
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver

## LLM Setup
Making some changes to simplify the LLM setup - adding a rate limiter so we can use GPT 4.1 and o3 without getting error rate failures.

In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.rate_limiters import InMemoryRateLimiter

rate_limiter = InMemoryRateLimiter(
    requests_per_second=4,  
    check_every_n_seconds=0.1,
    max_bucket_size=10,
)

In [3]:
from pathlib import Path

def load_yaml_config():
    with open(str((Path.cwd().parent / "conf.yaml").resolve()), "r") as f:
        config = yaml.safe_load(f)
        #processed_config = process_dict(config)
        return config

os.environ["OPENAI_API_KEY"] = load_yaml_config()['BASIC_MODEL']['api_key']
basic_llm = init_chat_model("openai:gpt-4.1-mini", rate_limiter=rate_limiter)
reasoning_llm = init_chat_model("openai:gpt-4.1-mini", rate_limiter=rate_limiter)

## Configuring State

Setting up the Planner and its fields

In [4]:
from pydantic import BaseModel, Field
from typing import List, Optional
from enum import Enum


class StepType(str, Enum):
    RESEARCH = "research"
    PROCESSING = "processing"

#used in Plan which is used in State
class Step(BaseModel):
    need_search: bool = Field(..., description="Must be explicitly set for each step")
    title: str
    description: str = Field(..., description="Specify exactly what data to collect")
    step_type: StepType = Field(..., description="Indicates the nature of the step")
    execution_res: Optional[str] = Field(
        default=None, description="The Step execution result"
    )

#used in the State object
class Plan(BaseModel):
    locale: str = Field(
        ..., description="e.g. 'en-US' or 'zh-CN', based on the user's language"
    )
    has_enough_context: bool
    thought: str
    title: str
    steps: List[Step] = Field(
        default_factory=list,
        description="Research & Processing steps to get more context",
    )

    class Config:
        json_schema_extra = {
            "examples": [
                {
                    "has_enough_context": False,
                    "thought": (
                        "To understand the current market trends in AI, we need to gather comprehensive information."
                    ),
                    "title": "AI Market Research Plan",
                    "steps": [
                        {
                            "need_search": True,
                            "title": "Current AI Market Analysis",
                            "description": (
                                "Collect data on market size, growth rates, major players, and investment trends in AI sector."
                            ),
                            "step_type": "research",
                        }
                    ],
                }
            ]
        }


In [5]:
from langgraph.graph import MessagesState
from pydantic import BaseModel, Field

#used in State
class Resource(BaseModel):
    """
    Resource is a class that represents a resource.
    """

    uri: str = Field(..., description="The URI of the resource")
    title: str = Field(..., description="The title of the resource")
    description: str | None = Field("", description="The description of the resource")

class State(MessagesState):
    """State for the agent system, extends MessagesState with next field."""

    # Runtime Variables
    locale: str = "en-US"
    research_topic: str = ""
    observations: list[str] = []
    resources: list[Resource] = []
    plan_iterations: int = 0
    current_plan: Plan | str = None
    final_report: str = ""
    auto_accepted_plan: bool = False
    enable_background_investigation: bool = True
    background_investigation_results: str = None


## Nodes

In [6]:
from langchain_core.runnables import RunnableConfig
from langgraph.types import Command, interrupt

In [7]:
def planner_node(
    state: State, config: RunnableConfig
) -> Command[Literal["human_feedback", "reporter"]]:
    """Planner node that generate the full plan."""
    logger.info("Planner generating full plan")
    configurable = Configuration.from_runnable_config(config)
    plan_iterations = state["plan_iterations"] if state.get("plan_iterations", 0) else 0
    messages = apply_prompt_template("planner", state, configurable)

    if state.get("enable_background_investigation") and state.get(
        "background_investigation_results"
    ):
        messages += [
            {
                "role": "user",
                "content": (
                    "background investigation results of user query:\n"
                    + state["background_investigation_results"]
                    + "\n"
                ),
            }
        ]

    plan_llm = reasoning_llm.with_structured_output(
            Plan,
            method="json_mode",
        )

    # if the plan iterations is greater than the max plan iterations, return the reporter node
    if plan_iterations >= configurable.max_plan_iterations:
        return Command(goto="reporter")

    full_response = ""
    response = plan_llm.invoke(messages)
    full_response = response.model_dump_json(indent=4, exclude_none=True)

    logger.debug(f"Current state messages: {state['messages']}")
    logger.info(f"Planner response: {full_response}")

    try:
        curr_plan = json.loads(repair_json_output(full_response))
    except json.JSONDecodeError:
        logger.warning("Planner response is not a valid JSON")
        if plan_iterations > 0:
            return Command(goto="reporter")
        else:
            return Command(goto="__end__")
    if curr_plan.get("has_enough_context"):
        logger.info("Planner response has enough context.")
        new_plan = Plan.model_validate(curr_plan)
        return Command(
            update={
                "messages": [AIMessage(content=full_response, name="planner")],
                "current_plan": new_plan,
            },
            goto="reporter",
        )
    return Command(
        update={
            "messages": [AIMessage(content=full_response, name="planner")],
            "current_plan": full_response,
        },
        goto="human_feedback",
    )

In [8]:
def reporter_node(state: State, config: RunnableConfig):
    """Reporter node that write a final report."""
    logger.info("Reporter write final report")
    configurable = Configuration.from_runnable_config(config)
    current_plan = state.get("current_plan")
    input_ = {
        "messages": [
            HumanMessage(
                f"# Research Requirements\n\n## Task\n\n{current_plan.title}\n\n## Description\n\n{current_plan.thought}"
            )
        ],
        "locale": state.get("locale", "en-US"),
    }
    invoke_messages = apply_prompt_template("reporter", input_, configurable)
    observations = state.get("observations", [])

    # Add a reminder about the new report format, citation style, and table usage
    invoke_messages.append(
        HumanMessage(
            content="IMPORTANT: Structure your report according to the format in the prompt. Remember to include:\n\n1. Key Points - A bulleted list of the most important findings\n2. Overview - A brief introduction to the topic\n3. Detailed Analysis - Organized into logical sections\n4. Survey Note (optional) - For more comprehensive reports\n5. Key Citations - List all references at the end\n\nFor citations, DO NOT include inline citations in the text. Instead, place all citations in the 'Key Citations' section at the end using the format: `- [Source Title](URL)`. Include an empty line between each citation for better readability.\n\nPRIORITIZE USING MARKDOWN TABLES for data presentation and comparison. Use tables whenever presenting comparative data, statistics, features, or options. Structure tables with clear headers and aligned columns. Example table format:\n\n| Feature | Description | Pros | Cons |\n|---------|-------------|------|------|\n| Feature 1 | Description 1 | Pros 1 | Cons 1 |\n| Feature 2 | Description 2 | Pros 2 | Cons 2 |",
            name="system",
        )
    )

    for observation in observations:
        invoke_messages.append(
            HumanMessage(
                content=f"Below are some observations for the research task:\n\n{observation}",
                name="observation",
            )
        )
    logger.debug(f"Current invoke messages: {invoke_messages}")
    response = llm.invoke(invoke_messages)
    response_content = response.content
    logger.info(f"reporter response: {response_content}")

    return {"final_report": response_content}

In [9]:
def human_feedback_node(
    state,
) -> Command[Literal["planner", "research_team", "reporter", "__end__"]]:
    current_plan = state.get("current_plan", "")
    # check if the plan is auto accepted
    auto_accepted_plan = state.get("auto_accepted_plan", False)
    if not auto_accepted_plan:
        feedback = interrupt("Please Review the Plan.")

        # if the feedback is not accepted, return the planner node
        if feedback and str(feedback).upper().startswith("[EDIT_PLAN]"):
            return Command(
                update={
                    "messages": [
                        HumanMessage(content=feedback, name="feedback"),
                    ],
                },
                goto="planner",
            )
        elif feedback and str(feedback).upper().startswith("[ACCEPTED]"):
            logger.info("Plan is accepted by user.")
        else:
            raise TypeError(f"Interrupt value of {feedback} is not supported.")

    # if the plan is accepted, run the following node
    plan_iterations = state["plan_iterations"] if state.get("plan_iterations", 0) else 0
    goto = "research_team"
    try:
        current_plan = repair_json_output(current_plan)
        # increment the plan iterations
        plan_iterations += 1
        # parse the plan
        new_plan = json.loads(current_plan)
        if new_plan["has_enough_context"]:
            goto = "reporter"
    except json.JSONDecodeError:
        logger.warning("Planner response is not a valid JSON")
        if plan_iterations > 0:
            return Command(goto="reporter")
        else:
            return Command(goto="__end__")

    return Command(
        update={
            "current_plan": Plan.model_validate(new_plan),
            "plan_iterations": plan_iterations,
            "locale": new_plan["locale"],
        },
        goto=goto,
    )

In [10]:
def research_team_node(state: State):
    """Research team node that collaborates on tasks."""
    logger.info("Research team is collaborating on tasks.")
    pass

## Runtime configuration

In [11]:
import enum

#used in configuration
class ReportStyle(enum.Enum):
    ACADEMIC = "academic"
    POPULAR_SCIENCE = "popular_science"
    NEWS = "news"
    SOCIAL_MEDIA = "social_media"


In [12]:
from dataclasses import dataclass, field, fields

#used in apply_prompt_templates
@dataclass(kw_only=True)
class Configuration:
    """The configurable fields."""

    resources: list[Resource] = field(
        default_factory=list
    )  # Resources to be used for the research
    max_plan_iterations: int = 1  # Maximum number of plan iterations
    max_step_num: int = 3  # Maximum number of steps in a plan
    max_search_results: int = 3  # Maximum number of search results
    mcp_settings: dict = None  # MCP settings, including dynamic loaded tools
    report_style: str = ReportStyle.ACADEMIC.value  # Report style

    @classmethod
    def from_runnable_config(
        cls, config: Optional[RunnableConfig] = None
    ) -> "Configuration":
        """Create a Configuration instance from a RunnableConfig."""
        configurable = (
            config["configurable"] if config and "configurable" in config else {}
        )
        values: dict[str, Any] = {
            f.name: os.environ.get(f.name.upper(), configurable.get(f.name))
            for f in fields(cls)
            if f.init
        }
        return cls(**{k: v for k, v in values.items() if v})

### To combine and access State and Config in the Nodes

In [19]:
from langgraph.prebuilt.chat_agent_executor import AgentState

def apply_prompt_template(
    prompt_name: str, state: AgentState, configurable: Configuration = None
) -> list:
    """
    Apply template variables to a prompt template and return formatted messages.

    Args:
        prompt_name: Name of the prompt template to use
        state: Current agent state containing variables to substitute

    Returns:
        List of messages with the system prompt as the first message
    """
    # Convert state to dict for template rendering
    state_vars = {
        "CURRENT_TIME": datetime.now().strftime("%a %b %d %Y %H:%M:%S %z"),
        **state,
    }

    # Add configurable variables
    if configurable:
        state_vars.update(dataclasses.asdict(configurable))

    try:
        template = env.get_template(f"templates/{prompt_name}.md")
        system_prompt = template.render(**state_vars)
        return [{"role": "system", "content": system_prompt}] + state["messages"]
    except Exception as e:
        raise ValueError(f"Error applying template {prompt_name}: {e}")


## Building the Graph

In [20]:
from jinja2 import Environment, FileSystemLoader, select_autoescape

# Initialize Jinja2 environment
env = Environment(
    loader=FileSystemLoader(os.path.dirname('.')),
    autoescape=select_autoescape(),
    trim_blocks=True,
    lstrip_blocks=True,
)

In [21]:
from langgraph.checkpoint.memory import MemorySaver
memory = MemorySaver()

graph_builder = StateGraph(State)
graph_builder.add_node("planner",planner_node)
graph_builder.add_node("reporter",reporter_node)
graph_builder.add_node("human_feedback",human_feedback_node)
graph_builder.add_node("research_team",research_team_node)
graph_builder.add_edge(START, "planner")
graph = graph_builder.compile(checkpointer=memory)

In [22]:
Mermaid(graph.get_graph(xray=True).draw_mermaid())

In [23]:
import logging
logger = logging.getLogger("node execution")
from datetime import datetime
import dataclasses

question = """What is the outlook for the cloud computing industry both in the west and emerging markets or regions?
What are some of the drivers of its growth and possible challenges or constraints?  What does the competitive landscape look like for leading
providers of cloud services?"""

initial_state = {
        #Setting up State
        "messages": [{"role": "user", "content": question}],
        "auto_accepted_plan": True,
        "enable_background_investigation": True,
    }

config = {
    #runtime configuration or variables
        "configurable": {
            "thread_id": "default",
            "max_plan_iterations": 1,
            "max_step_num": 3,
            "mcp_settings": {
                "servers": {
                    "mcp-github-trending": {
                        "transport": "stdio",
                        "command": "uvx",
                        "args": ["mcp-github-trending"],
                        "enabled_tools": ["get_github_trending_repositories"],
                        "add_to_agents": ["researcher"],
                    }
                }
            },
        },
        "recursion_limit": 100,
    }

last_message_cnt = 0
events = list(graph.stream(input=initial_state, config=config, stream_mode="values"))

In [24]:
events[1]['messages'][-1].pretty_print()

================================== Ai Message ==================================
Name: planner

{
    "locale": "en-US",
    "has_enough_context": false,
    "thought": "The user wants a comprehensive outlook on the cloud computing industry covering both western and emerging markets. They want detailed insights into growth drivers, challenges, constraints, and the competitive landscape among leading cloud providers. Given the multifaceted nature of the query covering geographic segmentation, market drivers, risks, and competition, and the need for abundant, up-to-date data points and analysis, the current context is insufficient and requires structured research. A well-rounded analysis should include historical and current market data, regional comparisons, detailed growth drivers and constraints, and a thorough competitive analysis involving key players and market trends.",
    "title": "Comprehensive Outlook and Competitive Analysis of the Cloud Computing Industry in Western and Emer

### Examining the state of the graph

In [25]:
snapshot = graph.get_state(config)
{k for k, v in snapshot.values.items()}

{'auto_accepted_plan',
 'current_plan',
 'enable_background_investigation',
 'locale',
 'messages',
 'plan_iterations'}

In [26]:
#understanding apply_prompt_template output
apply_prompt_template("planner", snapshot.values, Configuration.from_runnable_config(config))[0]['content'][:1000]

"---\nCURRENT_TIME: Fri Jun 20 2025 16:54:48 \n---\n\nYou are a professional Deep Researcher. Study and plan information gathering tasks using a team of specialized agents to collect comprehensive data.\n\n# Details\n\nYou are tasked with orchestrating a research team to gather comprehensive information for a given requirement. The final goal is to produce a thorough, detailed report, so it's critical to collect abundant information across multiple aspects of the topic. Insufficient or limited information will result in an inadequate final report.\n\nAs a Deep Researcher, you can breakdown the major subject into sub-topics and expand the depth breadth of user's initial question if applicable.\n\n## Information Quantity and Quality Standards\n\nThe successful research plan must meet these standards:\n\n1. **Comprehensive Coverage**:\n   - Information must cover ALL aspects of the topic\n   - Multiple perspectives must be represented\n   - Both mainstream and alternative viewpoints shoul

In [27]:
#graph steps and message collection
for state in graph.get_state_history(config):
    print("Num Messages: ", len(state.values["messages"]), "Next: ", state.next)
    print("-" * 80)

Num Messages:  2 Next:  ()
--------------------------------------------------------------------------------
Num Messages:  2 Next:  ('research_team',)
--------------------------------------------------------------------------------
Num Messages:  2 Next:  ('human_feedback',)
--------------------------------------------------------------------------------
Num Messages:  1 Next:  ('planner',)
--------------------------------------------------------------------------------
Num Messages:  0 Next:  ('__start__',)
--------------------------------------------------------------------------------
